<a href="https://colab.research.google.com/github/caiocezar400/DeepSea-Solutions/blob/main/Cinematica_direta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import sympy as sp
from sympy import symbols, cos, sin, pi, simplify, sympify

def dh_matrix(a, alpha, d, theta):
    """
    DH_MATRIX Denavit-Hartenberg
    Calcula a matriz de transformação homogênea 4x4
    """
    T = sp.Matrix([
        [cos(theta), -sin(theta)*cos(alpha),  sin(theta)*sin(alpha), a*cos(theta)],
        [sin(theta),  cos(theta)*cos(alpha), -cos(theta)*sin(alpha), a*sin(theta)],
        [0,           sin(alpha),             cos(alpha),            d],
        [0,           0,                      0,                     1]
    ])
    return T

def configurar_robo():
  # Solicita a configuração do robô (ex: RRR, RRP, RRRP)
  config_str = input('\nConfiguração do robô (ex: RRR, RRP, RP): ').strip().upper()

  # Remove caracteres inválidos se houver, mantendo apenas R ou P
  joints = [j for j in config_str if j in ['R', 'P']]
  N = len(joints)

  if N == 0:
    print("Erro: Nenhuma junta válida encontrada. Use 'R' para Revoluta ou 'P' para Prismática.")
    return

  print(f'\nRobô detectado com {N} juntas: {"-".join(joints)}')
  print('Para cada junta, insira os parâmetros constantes.')
  print('A variável da junta (q_i) será adicionada automaticamente.')
  print('(Pode usar valores simbólicos como L1, pi/2, d3, etc.)')

  dh_params = []

  for i, junta_tipo in enumerate(joints):
    idx = i + 1 # Índice da junta (1-based)

    # Cria a variável simbólica automática para esta junta (q1, q2, etc.)
    q_var = symbols(f'q{idx}')

    # Descrição do tipo de junta para o usuário
    tipo_desc = "Revoluta (R)" if junta_tipo == 'R' else "Prismática (P)"
    print(f'\n--- Junta {idx}: {tipo_desc} ---')

    # Inputs comuns a ambos os tipos
    a_str = input('a (cm): ').strip() or '0'
    alpha_str = input('alpha (rad): ').strip() or '0'

    # Converte para simbólico
    a = sympify(a_str)
    alpha = sympify(alpha_str)

    if junta_tipo == 'R':
      # Se é Revoluta: 'd' é constante, 'theta' é variável (q_i + offset)
      d_str = input('d (cm): ').strip() or '0'
      theta_offset_str = input('theta offset (rad) [Enter para 0]: ').strip() or '0'

      d = sympify(d_str)
      theta_offset = sympify(theta_offset_str)

      # Define theta como a variável q + o offset constante
      theta = q_var + theta_offset

      print(f'-> Configurado: theta = {theta}')

    elif junta_tipo == 'P':
      # Se é Prismática: 'theta' é constante, 'd' é variável (q_i + offset)
      theta_str = input('theta (rad): ').strip() or '0'
      d_offset_str = input('d offset (cm) [Enter para 0]: ').strip() or '0'

      theta = sympify(theta_str)
      d_offset = sympify(d_offset_str)

      # Define d como a variável q + o offset constante
      d = q_var + d_offset

      print(f'-> Configurado: d = {d}')

    # Converte entradas string em valores simbólicos
    params = [a, alpha, d, theta]
    dh_params.append(params)

  return dh_params, joints

def forward_kinematics_interactive():
    """
    Calculadora de cinemática direta interativa
    Suporta configuração automática baseada no tipo de junta (R ou P)
    """
    print("#################### Calculadora de cinemática direta ###############################")

    dh_params, _ = configurar_robo()
    N = len(dh_params)

    # Cria uma lista de todas as matrizes de transformação
    A = [dh_matrix(*params) for params in dh_params]

    # Calcula a matriz de transformação final
    T_total = sp.Identity(4)

    for matrix in A:
      T_total *= matrix

    # Define número de algoritmos significativos
    algsig = 6

    print('\nSimplificando a matriz final... (isso pode levar um momento)')

    # Tenta uma simplificação numérica inteligente (preserva pi, inteiros, mas reduz floats)
    T_total = simplify(T_total)

    # nsimplify converte floats muito pequenos para 0 e tenta encontrar representações racionais
    try:
        T_final = sp.nsimplify(T_total, tolerance=1e-10, rational=True)
    except:
        T_final = T_total.evalf(algsig)

    print(f'\nMatriz de transformação final (base para end-effector):\n')
    sp.pprint(T_final)

    print("\nLegenda:")
    print(f"q1..q{N} = Variáveis de junta")

# Main execution
if __name__ == "__main__":
    forward_kinematics_interactive()